In [11]:
import numpy as np
import matplotlib.pyplot as plt
import control
import ipywidgets as widgets
import sympy as sp
from IPython.display import display, Math, HTML

HTML("""
<style>
.widget-label, .custom-title {
    font-weight: bold !important;
    font-size: 16px !important;
}

.widget-toggle-buttons .widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
.widget-toggle-buttons .widget-toggle-button.selected {
    background-color: #007acc !important;
    color: white !important;
}

button.widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
button.widget-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
</style>
""")

In [12]:
s = sp.symbols('s')

def sympy_to_tf(expr, input_type="Step"):
    num, den = sp.fraction(sp.simplify(expr))
    num_poly = sp.Poly(num, s)
    den_poly = sp.Poly(den, s)

    # check for improper tf
    if num_poly.degree() > den_poly.degree():
        # polynomial divison
        q, r = sp.div(num, den, domain='QQ')
        poly_part = sp.simplify(q)
        rem_part = sp.simplify(r/den)

        display(Math(r"\textbf{Warning: The transfer function is IMPROPER "
                     "(degree numerator > degree denominator).}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 
       
        display(Math(r"\text{This means the system is non-causal and its impulse response "
                     "contains Dirac distributions } (\delta, \delta', \ldots)."))
        display(Math(r"\text{For simulation, only the strictly proper part is plotted.}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 
        
        display(Math(r"\bullet \ \text{Polynomial part (non-causal, NOT plotted): } " 
                     + sp.latex(poly_part)))
        display(Math(r"\bullet \ \text{Strictly proper part (plotted): } " 
                     + sp.latex(rem_part)))

        # if improper -> only proper is plotted
        num, den = sp.fraction(rem_part)
        num_poly = sp.Poly(num, s)
        den_poly = sp.Poly(den, s)


    if input_type == "Impulse":
        # check stricly proper tf
        if num_poly.degree() == den_poly.degree():
            q, r = sp.div(num, den, domain='QQ')
            const_part = sp.simplify(q)
            prop_part = sp.simplify(r/den)

            display(Math(r"\textbf{Warning: The transfer function has DIRECT FEEDTHROUGH (} D \neq 0 \textbf{).}"))
            display(HTML("<div style='margin:10px 0;'></div>")) 

            display(Math(r"\text{The impulse response contains a Dirac delta term that cannot be plotted.}"))
            display(HTML("<div style='margin:10px 0;'></div>")) 
            
            display(Math(r"\bullet \ \text{Feedthrough term (δ(t) contribution): } " + sp.latex(const_part)))
            display(Math(r"\bullet \ \text{Strictly proper part (plotted): } " + sp.latex(prop_part)))

            # only proper part is plotter
            num, den = sp.fraction(prop_part)
            num_poly = sp.Poly(num, s)
            den_poly = sp.Poly(den, s)
    

    num_coeffs = [float(coef) for coef in num_poly.all_coeffs()]
    den_coeffs = [float(coef) for coef in den_poly.all_coeffs()]
    return control.TransferFunction(num_coeffs, den_coeffs)

def calculate_specs(y, t):
    y = np.asarray(y).squeeze()
    t = np.asarray(t)

    # quick guards (diverging / oscillatory → no specs)
    if not np.all(np.isfinite(y)) or abs(y[-1]) > 1e6:
        return None
    if len(y) >= 100 and np.std(y[-100:]) > 0.05 * (abs(y[-1]) + 1e-12):
        return None

    y0  = float(y[0])
    yss = float(y[-1])
    A   = yss - y0
    rising = A >= 0

    y10 = y0 + 0.10 * A
    y90 = y0 + 0.90 * A

    t10 = t90 = None
    for i in range(len(y)):
        if t10 is None and ((y[i] >= y10) if rising else (y[i] <= y10)):
            t10 = float(t[i])
        if t90 is None and ((y[i] >= y90) if rising else (y[i] <= y90)):
            t90 = float(t[i])
        if t10 is not None and t90 is not None:
            break

   
    idx_peak = int(np.argmax(y)) if rising else int(np.argmin(y))
    peak     = float(y[idx_peak])
    t_peak   = float(t[idx_peak])

    # overshoot (%)
    eps = 1e-12
    denom = max(abs(yss), eps)
    overshoot = 0.0
    if rising:
        overshoot = max(0.0, (peak - yss) / denom * 100.0)
    else:
        overshoot = max(0.0, (yss - peak) / denom * 100.0)

    # settling time (±2% band)
    tol = 0.02 * max(abs(yss), 1.0)
    last_out = None
    for i in range(len(y)):
        if abs(y[i] - yss) > tol:
            last_out = i
    if last_out is None:
        Ts = float(t[0])
    elif last_out < len(t) - 1:
        Ts = float(t[last_out + 1])
    else:
        Ts = np.nan

    return {
        "SteadyStateValue": yss,
        "t_10": t10,
        "t_90": t90,
        "Peak": peak,
        "PeakTime": t_peak,
        "Overshoot": overshoot,
        "SettlingTime": Ts
    }


def plot_response(expr_input, input_type="Step", freq=1.0, t_end=10, show_specs = False):
    try:
        expr = sp.sympify(expr_input)
    except Exception as e:
        print("Error in the expression:", e)
        return

    try:
        system = sympy_to_tf(expr, input_type)
    except Exception as e:
        print("Error in the conversion in TransferFunction:", e)
        return

    t = np.linspace(0, t_end, 1000)

    if input_type == "Step":
        t, y = control.step_response(system, T=t)
        u = None
    elif input_type == "Impulse":
        t, y = control.impulse_response(system, T=t)
        u = None
    elif input_type == "Ramp":
        u = t
        t, y = control.forced_response(system, T=t, U=u)
    elif input_type == "Sine":
        u = np.sin(freq * t)
        t, y = control.forced_response(system, T=t, U=u)
    else:
        raise ValueError("Input not supported!")
    
    info = None

    # Plot
    plt.figure(figsize=(8,4))
    plt.plot(t, y, label="Output y(t)", color="tab:blue", linewidth=2)
    if u is not None:
        plt.plot(t, u, "--", label="Input u(t)", color="tab:orange", linewidth=1.5)

    info = calculate_specs(y, t) if (show_specs and input_type == "Step") else None
    if info is not None:
        y_ss = info["SteadyStateValue"]
        t10  = info["t_10"]
        t90  = info["t_90"]
        Ts   = info["SettlingTime"]

        # steady state
        plt.axhline(y_ss, linestyle=":", linewidth=1.2, label="Steady-State", color="green")
        # t10/t90 (hidden from legend)
        if t10 is not None:
            plt.axvline(t10, linestyle="--", linewidth=1.2, color="black", label="_nolegend_")
        if t90 is not None:
            plt.axvline(t90, linestyle="--", linewidth=1.2, color="black", label="_nolegend_")
        # settling time
        if Ts is not None and not np.isnan(Ts):
            plt.axvline(Ts, linestyle="-.", linewidth=1.2, label="Settling time", color="purple")

        # rise-time arrow @ 10% level
        if t10 is not None and t90 is not None:
            y0  = float(y[0])
            A   = y_ss - y0
            y10 = y0 + 0.10 * A
            ax = plt.gca()
            plt.annotate("", (t10, y10), (t90, y10),
                         arrowprops=dict(arrowstyle="<->", color="black"))
            plt.text((t10 + t90)/2,
                     y10 - 0.05*(ax.get_ylim()[1]-ax.get_ylim()[0]),
                     "Rise time", ha="center", va="top")

        # overshoot arrow (one-sided, only if meaningful)
        if info["Overshoot"] > 1e-3:
            peak   = info["Peak"]
            t_peak = info["PeakTime"]
            plt.annotate("", (t_peak, y_ss), (t_peak, peak),
                         arrowprops=dict(arrowstyle="<-", color="black", lw=1.5))
            plt.text(t_peak + 0.2, (peak + y_ss)/2,
                     r"Overshoot $M_p$", ha="left", va="center")
                    
    plt.title(f"System Response", fontsize=14)
    plt.xlabel("Time [s]", fontsize=12)
    plt.ylabel("Output y(t)")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.legend()
    plt.show()

style_bold = {'description_width': 'initial'}
layout_box = widgets.Layout(width="500px", height="70px")

expr_text = widgets.Textarea(
    value="(s+1)/(s+2)",
    description="Transfer Function:",
    layout=layout_box,
    style=style_bold
)

show_specs_box = widgets.ToggleButton(
    value = False, 
    description = "Show Time-Domain specifications",
    button_style='success',
    layout=widgets.Layout(width="265px", height="40px")

)


input_buttons = widgets.ToggleButtons(
    options=["Step", "Impulse", "Ramp", "Sine"],
    value="Step",
    description="Input:",
    button_style="info",
    tooltips=["Step", "Impulse", "Ramp", "Sine"],
    style={'description_width': 'initial'}
)

freq_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=10.0, step=0.1, description="f:"
)
tend_slider = widgets.FloatSlider(
    value=10, min=1, max=100, step=1, description="T:"
)

freq_label = widgets.HTML(value="<span class='custom-title'>Frequency (only for Sine input):</span>")
time_label = widgets.HTML(value="<span class='custom-title'>Time:</span>")

reset_button = widgets.Button(description="Reset", button_style="danger", layout=widgets.Layout(width="200px", height="40px"))

def reset_values(b):
    expr_text.value = "(s+1)/(s+2)"
    input_buttons.value = "Step"
    freq_slider.value = 1.0
    tend_slider.value = 10
    show_specs_box.value = False

reset_button.on_click(reset_values)

ui = widgets.VBox([expr_text, input_buttons, freq_label, freq_slider, time_label, tend_slider, show_specs_box, reset_button])

out = widgets.interactive_output(
    lambda expr_input, input_type, freq, t_end, show_specs: plot_response(
        expr_input, input_type, freq, t_end, show_specs
    ),
    {
        "expr_input": expr_text,
        "input_type": input_buttons,
        "freq": freq_slider,
        "t_end": tend_slider, 
        "show_specs": show_specs_box
    }
)

display(ui, out)

Output()